# Average Bubble

In [1]:
from bubble_tools import *
from experiment import *


/home/dpirvu/.local/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [2]:
import matplotlib.pyplot as plt

plt.rcParams.update({'backend' : 'Qt5Agg'})
plt.rcParams.update({'text.usetex' : True})

plt.rcParams.update({'font.size' : 12.0})
plt.rcParams.update({'axes.titlesize' : 14.0})  # Font size of title
plt.rcParams.update({'axes.titlepad'  : 10.0})
plt.rcParams.update({'axes.labelsize' : 14.0})  # Axes label sizes
plt.rcParams.update({'axes.labelpad'  : 10.0})
plt.rcParams.update({'xtick.labelsize'  : 14.0})
plt.rcParams.update({'ytick.labelsize'  : 14.0})
plt.rcParams.update({'xtick.labelsize'  : 10.0})
plt.rcParams.update({'ytick.labelsize'  : 10.0})

plt.rcParams.update({'axes.spines.left'  : True})
plt.rcParams.update({'axes.spines.right'  : True})
plt.rcParams.update({'axes.spines.top'  : True})
plt.rcParams.update({'axes.spines.bottom'  : True})
plt.rcParams.update({'savefig.format'     : 'pdf'})
plt.rcParams.update({'savefig.bbox'       : 'tight'})
plt.rcParams.update({'savefig.pad_inches' : 0.1})
plt.rcParams.update({'pdf.compression' : 6})
#plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#001219', '#005f73', '#0a9396', '#94d2bd', '#e9d8a6', '#ee9b00', '#ca6702', '#bb3e03', '#ae2012', '#9b2226'])

def ax_opts(ax):
    ax[0].axhline(phieq, color='darkgray', ls='--')
    ax[0].axhline(2.*np.pi, color='darkgray', ls='--') 
    ax[1].axvline(phieq, color='darkgray', ls='--')
    ax[1].axvline(2.*np.pi, color='darkgray', ls='--') 
    ax[0].yaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
    ax[0].yaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
    ax[1].xaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
    ax[1].xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
    ax[0].set_xlabel(r'$r$')
    ax[0].set_ylabel(r'$\phi$')
    ax[1].set_ylabel(r'$r$')
    ax[1].set_xlabel(r'$\phi$')
    for axx in ax:
        axx.legend()
        axx.grid()
    return ax

In [3]:
bubbleList, instantonList, tmpLoaded, indList = [], [], [], []
for tmp, temp in enumerate(tempList):
    lamb     = lambList[tmp]
    sigmafld = fluct_stdev(lamb, phi0, temp)
    pathavbub = average_bubble_file(nLat, lamb, phi0, temp)+'.npy'
    bubbleList.append(np.load(pathavbub))

    instantonList.append(np.load(path_inst(nLat, lamb, phi0, temp)+'.npy'))

    tmpLoaded.append(tmp)

    indList.append(np.load(path_indcrit(nLat, lamb, phi0, temp)+'.npy'))

NameError: name 'tempList' is not defined

In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (15, 7))
cols = cycle(['orange', 'darkgreen', 'violet', 'orangered'])

fvrecoList = []
for ii, average_bubble in enumerate(bubbleList):

    indmin = int(indList[ii])
    tmp = tmpLoaded[ii]
    temp = tempList[tmp]
    lamb = lambList[tmp]

    col = next(cols)
    xmin = 100

    data   = average_bubble[0,0,indmin,xmin:-xmin]
   # if temp==0: data = gaussian_filter(data, 3, mode='nearest')
    data = gaussian_filter(data, 1, mode='nearest')

    xpeak  = np.argmax(data)+1
    data   = data[:xpeak]

    coords = data>phieq*1.01
    xstart = np.argwhere(data*coords > 0.)[0][0]; print(xstart)
    xstart = 100

    data   = data[xstart:xpeak+1]
    xlist  = np.linspace(xstart, xpeak, len(data))*dx

    spline     = scp.interpolate.splrep(xlist, data)    # interpolate mean field
    der1spline = scp.interpolate.splder(spline, n=1)    # take tangent to this curve
    der2spline = scp.interpolate.splder(spline, n=2)    # take tangent to tangent curve

    r        = np.linspace(xlist[0], xlist[-1], 1000)
    phifv    = scp.interpolate.splev(r, spline)
    dphidr   = scp.interpolate.splev(r, der1spline)
    ddphiddr = scp.interpolate.splev(r, der2spline)

    # invert \phi(r) into r(\phi)
    rinv = np.zeros(phifv.shape)
    profile = intp.interp1d(r, phifv, kind='linear')
    diff = lambda x, a: (profile(x) - a)**2.
    for idx, x_value in enumerate(phifv):
        res = sco.minimize_scalar(diff, args=(x_value), bounds=(min(r), max(r)), method='Bounded')
        rinv[idx] = res.x

    # integrate product of derivatives above to get potential function of r
    if temp==0:
        dVdr = scp.interpolate.splrep(r, dphidr*ddphiddr + dphidr**2./r)
    else:
        dVdr = scp.interpolate.splrep(r, dphidr*ddphiddr)
    Vofrofphi = scp.interpolate.splev(rinv, scp.interpolate.splantider(dVdr, n=1))
    fvrecoList.append(np.asarray([tmp, temp, lamb, phi0, r, phifv, dphidr, ddphiddr, Vofrofphi]))

    labs = r'$\phi_0=$'+str('%.2f'%phi0)+r', $\lambda=$'+str(lamb)+r', $T=$'+str(temp)
    lsl = ('-' if tmp==1 else '--' if tmp==0 else ':' if tmp==2 else '-.')

    ax[0].plot(xlist, data , color=col, ls=lsl, marker='o', ms=2)
    ax[0].plot(r,     phifv, color=col, ls=lsl, label=labs)
    ax[1].plot(phifv, rinv , color=col, ls=lsl, label=labs)

ax = ax_opts(ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (13, 8))
for ii, lists in enumerate(fvrecoList):
    tmp, temp, lamb, phi0, r, phifv, dphidr, ddphiddr, Vofrofphi = lists

    lsl = ('-' if tmp==1 else '--' if tmp==0 else ':' if tmp==2 else '-.')

    ax[0].plot(r, phifv-phieq, 'r', ls=lsl, label=(f'$\phi(r)$' if ii==0 else None))
    ax[0].plot(r, dphidr     , 'b', ls=lsl, label=(f'$\partial_r \phi(r)$' if ii==0 else None))
    ax[0].plot(r, ddphiddr   , 'g', ls=lsl, label=(f'$\partial_r^2 \phi(r)$' if ii==0 else None))
    ax[1].plot(r, Vofrofphi  , 'g', ls=lsl, label=(f'$V(r)$' if ii==0 else None))

for axx in ax: axx.set_xlabel(r'$r$'); axx.legend(); axx.grid()
plt.show()


# The Roll 

In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (15, 7))
cols = cycle(['orange', 'darkgreen', 'violet', 'orangered'])

rollrecoList = []
for ii, average_bubble in enumerate(bubbleList):
    indmin = int(indList[ii])
    tmp = tmpLoaded[ii]
    temp = tempList[tmp]
    lamb = lambList[tmp]

    col = next(cols)
    xpeak  = np.argmax(average_bubble[0,0,indmin])
    data  = average_bubble[0,0][indmin+1:, xpeak]
   # if temp==0: data = gaussian_filter(data, 3, mode='nearest')
    data = gaussian_filter(data, 1, mode='nearest')

    tlist = np.linspace(1, len(data), len(data))*dtout

    tstop = np.argmax(data)+1
    data  = data[:tstop+1]
    tlist = tlist[:tstop+1]

    spline     = scp.interpolate.splrep(tlist, data)    # interpolate mean field
    der1spline = scp.interpolate.splder(spline, n=1)    # take tangent to this curve
    der2spline = scp.interpolate.splder(spline, n=2)    # take tangent to tangent curve

    rt        = np.linspace(tlist[0], tlist[-1], 1000)
    phiroll    = scp.interpolate.splev(rt, spline)
    dphidr   = scp.interpolate.splev(rt, der1spline)
    ddphiddr = scp.interpolate.splev(rt, der2spline)

    # invert \phi(r) into r(\phi)
    tinv = np.zeros(phiroll.shape)
    profile = intp.interp1d(rt, phiroll, kind='linear')
    difft = lambda x, a: (profile(x) - a)**2.
    for idx, x_value in enumerate(phiroll):
        res = sco.minimize_scalar(difft, args=(x_value), bounds=(min(rt), max(rt)), method='Bounded')
        tinv[idx] = res.x

    # integrate product of derivatives above to get potential function of r
    if temp==0:
        dVdr = scp.interpolate.splrep(rt, - dphidr*ddphiddr - dphidr**2./rt )
    else:
        dVdr = scp.interpolate.splrep(rt, - dphidr*ddphiddr - dphidr**2./rt )
    Vofrofphi = scp.interpolate.splev(tinv, scp.interpolate.splantider(dVdr, n=1))
    rollrecoList.append(np.asarray([tmp, temp, lamb, phi0, rt, phiroll, dphidr, ddphiddr, Vofrofphi]))

    labs = r'$\phi_0=$'+str('%.2f'%phi0)+r', $\lambda=$'+str(lamb)+r', $T=$'+str(temp)
    lsl = ('-' if tmp==1 else '--' if tmp==0 else ':' if tmp==2 else '-.')

    ax[0].plot(tlist, data , color=col, ls='None', marker='o', ms=2)
    ax[0].plot(rt, phiroll, color=col, ls=lsl, label=labs)
    ax[1].plot(phiroll, tinv , color=col, ls=lsl, label=labs)

ax = ax_opts(ax)
plt.show()


In [ ]:
fig, ax = plt.subplots(2, 1, figsize = (13, 8))
for ii, lists in enumerate(rollrecoList):
    tmp, temp, lamb, phi0, ts, phiroll, dphidr, ddphiddr, Vofrofphi = lists

    lsl = ('-' if tmp==1 else '--' if tmp==0 else ':' if tmp==2 else '-.')

    ax[0].plot(ts, phiroll-phieq               , 'r', ls=lsl, label=(f'$\phi(r)$' if ii==0 else None))
    ax[0].plot(ts, dphidr                      , 'b', ls=lsl, label=(f'$\partial_r \phi(r)$' if ii==0 else None))
    ax[0].plot(ts, ddphiddr                    , 'g', ls=lsl, label=(f'$\partial_r^2 \phi(r)$' if ii==0 else None))
    ax[1].plot(ts, Vofrofphi-np.mean(Vofrofphi), 'g', ls=lsl, label=(f'$V(r)$' if ii==0 else None))
    ax[1].plot(ts, -dphidr*ddphiddr            , 'r', ls=lsl, label=(f'$d\phi/drd^2\phi/dr^2$' if ii==0 else None))
    ax[1].plot(ts, dphidr**2./ts               , 'b', ls=lsl, label=(f'$1/r d\phi/dr$' if ii==0 else None))

for axx in ax: axx.set_xlabel(r'$r$'); axx.legend(); axx.grid()
plt.show()

# Full reconstruction

In [ ]:
fullRec = []
for ii in range(len(rollrecoList)):
    tmp, temp, lamb, phi0, ts, phiroll, dphidr, ddphiddr, Vrollofrofphi = rollrecoList[ii]
    tmp, temp, lamb, phi0, r, phifv, dphidr, ddphiddr, Vfvofrofphi = fvrecoList[ii]

    fullRec.append(np.asarray([np.concatenate((phifv, phiroll)), np.concatenate((Vfvofrofphi, Vrollofrofphi))-V(2.*np.pi, lamb)]))


In [ ]:
def fit_to_V(x, lam, V0, a):
    return (-np.cos(x) + 0.5*lam**2. * np.sin(x)**2. + a) * V0

fig, ax = plt.subplots(1, 1, figsize = (10, 5))

cols = cycle(['orange', 'darkgreen', 'violet', 'orangered'])
extphi = np.linspace(phieq*0.8, 2.*np.pi*1.1, 100)
for ii in range(len(rollrecoList)):
    if ii==0: continue
    col = next(cols)
    
    tmp, temp, lamb, phi0, ts, phiroll, dphidr, ddphiddr, Vrollofrofphi = rollrecoList[ii]
    tmp, temp, lamb, phi0, r, phifv, dphidr, ddphiddr, Vfvofrofphi = fvrecoList[ii]
    prediction = np.concatenate((V(phifv, lamb), V(phiroll, lamb))) - V(np.pi, lamb)
    xlist = np.concatenate((phifv, phiroll))
    curve = fullRec[ii][1]
    
    lsl = ('-' if tmp==1 else '--' if tmp==0 else ':' if tmp==2 else '-.')

    if ii==1:
        plt.plot(xlist, prediction, ls='-', linewidth=1, alpha=0.8, color='orangered', label='Bare potential')

    plt.plot(xlist, curve, color=col, ls=lsl, alpha=2)

    popt, _ = scp.optimize.curve_fit(fit_to_V, xlist, curve)
    plt.plot(extphi, fit_to_V(extphi, *popt), color=col, ls=lsl, label=f'$V(\phi, \lambda=%5.2f)$'%(popt[0]), linewidth=1, alpha=1)

    spline    = scp.interpolate.splrep(extphi, fit_to_V(extphi, *popt))
    seconder  = scp.interpolate.splev(phieq, scp.interpolate.splder(spline, n=2))
    print('lambda =', lamb, m2(lamb), ', fit params:', *popt, r', $m2(\lambda) = $', seconder)

plt.legend()
plt.grid(alpha=0.5)
plt.xlabel(r'$\phi$')
plt.ylabel(r'$V(\phi;\lambda)$')
ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
ax.xaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
plt.savefig('./data/reconstructed_potential.pdf', rasterize=True)
plt.show()